# Aula 5
## Desafio 1
___

Dado o dataset de resultados das eleições de 2020, obtenha a lista de candidatos (nome) que obtiveram a menor votação em cada cidade do país.

* A tabela resultante deve conter:
* Estado
* Município
* Nome do candidato
* Quantidade de votos


In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=6d3bb9883659ac0ffd966c6e78e3a56f61bccb6677bcd2f3c943d1a5d38d1041
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .master('local[*]') \
  .appName("Analise de dados de eleições") \
  .config('spark.ui.port', '4050') \
  .getOrCreate()

In [3]:
!git clone https://github.com/michelpf/dataset-brazil-elections-2020-mayor-1st-round

Cloning into 'dataset-brazil-elections-2020-mayor-1st-round'...
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 326 (delta 0), reused 3 (delta 0), pack-reused 323
Receiving objects: 100% (326/326), 127.02 MiB | 15.58 MiB/s, done.
Resolving deltas: 100% (262/262), done.
Updating files: 100% (151/151), done.


In [4]:
dados = spark.read.option("encoding", "iso-8859-1").option("recursiveFileLookup","true").csv('dataset-brazil-elections-2020-mayor-1st-round/dataset', sep=';', header=True, inferSchema=True)

In [6]:
from pyspark.sql import functions as f

In [7]:
dados_mun_votacao = dados\
  .select("SG_UF", "NM_MUNICIPIO", "NM_VOTAVEL", "QT_VOTOS")\
  .where(~f.col("NM_VOTAVEL").isin('Nulo','Branco'))\
  .groupBy("SG_UF", "NM_MUNICIPIO", "NM_VOTAVEL")\
  .agg(
      f.sum("QT_VOTOS").alias("SOMA_VOTOS")
  )\
  .orderBy("NM_MUNICIPIO")

dados_mun_votacao.show()

+-----+-------------------+-------------------+----------+
|SG_UF|       NM_MUNICIPIO|         NM_VOTAVEL|SOMA_VOTOS|
+-----+-------------------+-------------------+----------+
|   GO|    ABADIA DE GOIÁS|ITAMAR MARKA DA PAZ|       532|
|   GO|    ABADIA DE GOIÁS|      ZÉ DA PATROLA|      2226|
|   GO|    ABADIA DE GOIÁS|        IVAN JARDIM|        56|
|   GO|    ABADIA DE GOIÁS|     WANDER SARAIVA|      5053|
|   MG|ABADIA DOS DOURADOS|    WANDERLEI LEMES|      2642|
|   MG|ABADIA DOS DOURADOS| VAGNER DA ABAPEVEL|      2044|
|   GO|          ABADIÂNIA|           ZÉ DINIZ|      4503|
|   GO|          ABADIÂNIA|      PAULO PAULADA|        36|
|   GO|          ABADIÂNIA|      WENDER CHAVES|       631|
|   GO|          ABADIÂNIA| TENENTE ALESSANDRO|      2994|
|   PA|         ABAETETUBA|              CHITA|     20615|
|   PA|         ABAETETUBA|  ADAMOR BITENCOURT|     12867|
|   PA|         ABAETETUBA|     EVERSON CARLOS|     13995|
|   PA|         ABAETETUBA|     PEDRO HENRIQUE|     1768

In [8]:
dados_mun_min_votacao = dados_mun_votacao\
  .select("SG_UF", "NM_MUNICIPIO", "SOMA_VOTOS")\
  .groupBy("SG_UF", "NM_MUNICIPIO")\
  .agg(
      f.min("SOMA_VOTOS").alias("SOMA_VOTOS")\
  )\
  .orderBy("NM_MUNICIPIO")

dados_mun_min_votacao.show()

+-----+-------------------+----------+
|SG_UF|       NM_MUNICIPIO|SOMA_VOTOS|
+-----+-------------------+----------+
|   GO|    ABADIA DE GOIÁS|        56|
|   MG|ABADIA DOS DOURADOS|      2044|
|   GO|          ABADIÂNIA|        36|
|   PA|         ABAETETUBA|     12867|
|   MG|             ABAETÉ|       286|
|   CE|            ABAIARA|      2584|
|   BA|              ABARÉ|      4521|
|   PR|             ABATIÁ|       393|
|   BA|             ABAÍRA|      2284|
|   SC|      ABDON BATISTA|      1093|
|   PA|    ABEL FIGUEIREDO|      2248|
|   SC|       ABELARDO LUZ|       654|
|   MG|         ABRE CAMPO|      2722|
|   PE|       ABREU E LIMA|       176|
|   TO|        ABREULÂNDIA|       894|
|   MG|            ACAIACA|        46|
|   BA|          ACAJUTIBA|      3602|
|   CE|            ACARAPE|        16|
|   CE|             ACARAÚ|        33|
|   RN|              ACARI|      3143|
+-----+-------------------+----------+
only showing top 20 rows



In [9]:
dados_mun_min_votacao_cand =  dados_mun_votacao.alias("a") \
  .join(
      dados_mun_min_votacao.alias("b"),
      (dados_mun_votacao.SG_UF == dados_mun_min_votacao.SG_UF) &
      (dados_mun_votacao.NM_MUNICIPIO == dados_mun_min_votacao.NM_MUNICIPIO) &
      (dados_mun_votacao.SOMA_VOTOS == dados_mun_min_votacao.SOMA_VOTOS),
      how="INNER") \
  .orderBy(f.col("b.NM_MUNICIPIO"), f.col("a.SG_UF")) \
  .select(f.col("a.SG_UF"), f.col("a.NM_MUNICIPIO"), f.col("a.NM_VOTAVEL"), f.col("b.SOMA_VOTOS"))
dados_mun_min_votacao_cand.show(truncate=False)

+-----+-------------------+-----------------------------+----------+
|SG_UF|NM_MUNICIPIO       |NM_VOTAVEL                   |SOMA_VOTOS|
+-----+-------------------+-----------------------------+----------+
|GO   |ABADIA DE GOIÁS    |IVAN JARDIM                  |56        |
|MG   |ABADIA DOS DOURADOS|VAGNER DA ABAPEVEL           |2044      |
|GO   |ABADIÂNIA          |PAULO PAULADA                |36        |
|PA   |ABAETETUBA         |ADAMOR BITENCOURT            |12867     |
|MG   |ABAETÉ             |FERNANDO PEIXINHO            |286       |
|CE   |ABAIARA            |PEDRO MARCOS                 |2584      |
|BA   |ABARÉ              |TATI OLIVEIRA                |4521      |
|PR   |ABATIÁ             |TONINHO ROSA                 |393       |
|BA   |ABAÍRA             |JOÃO HIPÓLITO RODRIGUES FILHO|2284      |
|SC   |ABDON BATISTA      |ABELHA FILHO                 |1093      |
|PA   |ABEL FIGUEIREDO    |HIDELFONSO                   |2248      |
|SC   |ABELARDO LUZ       |TIAGO K